<a href="https://colab.research.google.com/github/palexey2006/Computer-vision-projects/blob/main/PytorchExtra/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torchvision
import ssl
import torchvision.transforms as transforms
from tqdm import tqdm

import torch.nn as nn
ssl._create_default_https_context = ssl._create_unverified_context


class Identity(nn.Module):
    def __init__(self):
        super(Identity,self).__init__()

    def forward(self, x):
        return x


# loading pretrained model

model = torchvision.models.vgg16(pretrained=True)
#print(model)
# to freeze previous layers we can use for loop
# freezing previous layers helps a model to learn quicker
for param in model.parameters():
    param.requires_grad = False
#print('after modifying the model')
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512,100),
    nn.Dropout(0.5),
    nn.Linear(100,10),
)
#print(model)

#Hyperparams

learning_rate = 0.001
epochs = 2
batch_size = 64
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(train_loader)


for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    for step, (images, labels) in enumerate(loop):
        images = images
        labels = labels
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_description(f'Epoch [{epoch + 1}/{epochs}]')
        loop.set_postfix(loss=loss.item())
    print('')


model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'accuracy: {accuracy:.4f}')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/2]: 100%|██████████| 782/782 [05:49<00:00,  2.24it/s, loss=1.58]


Epoch [2/2]: 100%|██████████| 782/782 [05:57<00:00,  2.18it/s, loss=0.979]



accuracy: 60.1300
